In [2]:
import pandas as pd
import numpy as np
import sys
import os
import re

In [3]:
dot_csv = re.compile('[0-9]{4}_[0-9]{2}[.]csv')
csv_files = [file for file in os.listdir("traceset1") if dot_csv.match(file)]

In [5]:
def split_file(filename):
    df = pd.read_csv("traceset1/"+filename)
    strt = 0
    prev_time = pd.to_datetime(df.loc[0,'timestamp'])
    for i,time in enumerate(df['timestamp']):
        cur_time = pd.to_datetime(time)
        if(prev_time.day != cur_time.day):
            df[strt:i].to_csv("trace_by_date/"+prev_time.strftime("%Y_%m_%d")+".csv")
            prev_time = cur_time
            strt = i
    df[strt:].to_csv("trace_by_date/"+prev_time.strftime("%Y_%m_%d")+".csv")

In [6]:
for file in csv_files:
    split_file(file)

KeyboardInterrupt: 